In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, TensorDataset
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Set de datos
Antes de armar la red neuronal vamos a importar los datos que se van a requerir usando pandas. Vamos a hacer un preprocesamiento para que el formato sea compatible con la red neuronal que armaremos

In [ ]:
datos = pd.read_csv("Churn_Modelling.csv")
datos.head()

Imprimiento los primeros 5 elementos podemos ver que nuestra variable destino es la ultima columna **Exited** donde 1 es que se dio de baja y 0 que se quedó.

Es importante que en este paso vayamos pensando que tendremos que hacer con algunas columnas, ya sea eliminarlas, normalizarlas, llevar alguna transformación etc. Por ejemplo:
* RowNumber, CustomerID, Surname no nos dicen nada de utilidad. Predecir si alguien va a darse de baja por su ID de cliente, apellido o numero de renglon en la base de datos no nos va a dar buenos resultados. 
* Las columnas que son categoricas las vamos a transformar con **one hot encoding** (Genero y ubicacion geográfica)

In [ ]:
# Separamos la ultima columna para que sea variable destino
datos_y = datos[datos.columns[-1]]
datos_y.head()


In [ ]:
# Eliminamos las columnas que no funcionarán
datos_x = datos.drop(columns=["RowNumber", "CustomerId", "Surname", "Exited"])
datos_x.head()

In [ ]:
# Convertimos en one hot encoding las columnas de genero y zona geográfica
datos_x = pd.get_dummies(datos_x)
datos_x.head()

Ya se cuenta con una variable que tiene todas las entradas al modelo **datos_x** y otra con la salida **datos_y**. Para datos_x se cuenta con una columna por cada categoria de las variables Genero y Zona geofráfica.


# Escalado datos
Ahora vamos a escalar los valores para que esten dentro de un rango mas corto.

In [ ]:
escalador = StandardScaler()
datos_x = escalador.fit_transform(datos_x)
print(datos_x)

# Dividir datos entre entrenamiento y test

In [ ]:
datos_x.shape[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(datos_x, datos_y, test_size = 0.2, random_state = 2)

In [ ]:
print("X Train: {}, X Test: {}, y_train: {}, y_test: {}".format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

In [ ]:
n_entradas = X_train.shape[1]

# Tensores
Para poder procesar los datos en la red neuronal necesitamos que todos los datos estén en tensores, asi que haremos las conversiones necesarias

In [ ]:
t_X_train = torch.from_numpy(X_train).float().to("cpu") #MPS
t_X_test = torch.from_numpy(X_test).float().to("cpu")
t_y_train = torch.from_numpy(y_train.values).float().to("cpu")
t_y_test = torch.from_numpy(y_test.values).float().to("cpu")
t_y_train = t_y_train[:,None]
t_y_test = t_y_test[:, None]


In [ ]:
test = TensorDataset(t_X_test, t_y_test)
print(test[0])

In [ ]:
t_y_train

# Estructura de la red neuronal
Ahora vamos a armar una estructura básica de una red neuronal la cual va a recibir los datos de **X** para eventualmente poder predecir **y**

Para hacer esto tenemos que crear una Clase la cual hereda de nn.Module de torch.

In [ ]:
class Red(nn.Module):
    
    def __init__(self, n_entradas):
        super(Red, self).__init__()
        self.linear1 = nn.Linear(n_entradas, 15)
        self.linear2 = nn.Linear(15, 8)
        # self.linear3 = nn.Linear(8,160)
        # self.linear4 = nn.Linear(160, 200)
        # self.linear5 = nn.Linear(200, 1)
        self.linear3 = nn.Linear(8, 1)
    
    def forward(self, inputs):
        pred_1 = torch.sigmoid(input=self.linear1(inputs))
        pred_2 = torch.sigmoid(input=self.linear2(pred_1))
        # prediction = torch.sigmoid(input=self.linear3(prediction))
        # prediction = torch.sigmoid(input=self.linear4(prediction))
        # prediction = torch.sigmoid(input=self.linear5(prediction))
        pred_f = torch.sigmoid(input=self.linear3(pred_2))
        return pred_f


In [ ]:
t_y_test[0]

In [ ]:
%%time
lr = 0.001
epochs = 2000
estatus_print = 100

model = Red(n_entradas=n_entradas)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
print("Arquitectura del modelo: {}".format(model))
historico = pd.DataFrame()

print("Entranando el modelo")
for epoch in range(1, epochs+1):
    y_pred= model(t_X_train)
    loss = loss_fn(input=y_pred, target=t_y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % estatus_print == 0:
        print(f"\nEpoch {epoch} \t Loss: {round(loss.item(), 4)}")
    
    with torch.no_grad():
        y_pred = model(t_X_test)
        y_pred_class = y_pred.round()
        correct = (y_pred_class == t_y_test).sum()
        accuracy = 100 * correct / float(len(t_y_test))
        if epoch % estatus_print == 0:
            print("Accuracy: {}".format(accuracy.item()))
    
    df_tmp = pd.DataFrame(data={
        'Epoch': epoch,
        'Loss': round(loss.item(), 4),
        'Accuracy': round(accuracy.item(), 4)
    }, index=[0])
    historico = pd.concat(objs=[historico, df_tmp], ignore_index=True, sort=False)

print("Accuracy final: {}".format(round(accuracy.item(), 4)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10, 10))
plt.plot(historico['Epoch'], historico['Loss'], label='Loss')
plt.title("Loss", fontsize=25)
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Loss", fontsize=12)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(historico['Epoch'], historico['Accuracy'], label='Accuracy')
plt.title("Accuracy", fontsize=25)
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)
plt.grid()
plt.show()

In [ ]:
t_X_test[5]

In [ ]:
prediccion = model(t_X_test[5])
print(prediccion)

In [ ]:
t_y_test[5]